In [ ]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import SnowballStemmer

In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/train_content.csv')

In [ ]:
data.head()

,_id,date,tags,title,hubs_title,description,name,hub,png,nick,url,content,favs_lognorm
0,https://geektimes.ru/post/21866/,2008-03-17T18:55:00.000Z,"['eeepc', 'asus', 'ЭТО', 'эльдорадо', 'ура']",eeePC в продаже. Да. Правда.,Железо,"Итак, если 3 дня назад я отписался то что в пр...",Сергей 'pokatusher',hub/hardware,https://habrastorage.org/getpro/habr/olpicture...,@M_org,https://geektimes.ru/users/M_org,"Итак, если 3 дня назад я <a href=""http://habra...",2.484907
1,https://habrahabr.ru/company/aladdinrd/blog/30...,2016-06-24T13:02:00.000Z,"['Интеграция', 'шифрование', 'Windows', 'Win32...",«Разрубить Гордиев узел» или преодоление пробл...,Системное программирование,Современная операционная система это сложный и...,Аладдин Р.Д.,hub/system_programming,https://habrastorage.org/files/cbd/cf9/5ff/cbd...,NaN,https://habrahabr.ru/company/aladdinrd,Современная операционная система это сложный и...,4.174387
2,https://geektimes.ru/post/92887/,2010-05-06T10:00:00.000Z,"['mc', 'midnight commander', 'diffview', 'merg...",Релиз Midnight Commander 4.7.2 и 4.7.0.5,Чёрная дыра,Спустя 2 месяца упорных трудов вышла новая вер...,Илья Маслаков,hub/closet,https://geektimes.ru/images/logo.png,@smind,https://geektimes.ru/users/smind,Спустя 2 месяца упорных трудов вышла новая вер...,0.000000
3,https://habrahabr.ru/post/290824/,2015-05-22T11:01:00.000Z,"['бизнес-модель', 'бизнес-моделирование']",7 шагов для постройки правильной бизнес-модели,Интернет-маркетинг,Большинство IT предпринимателей сосредотачиваю...,Александр,hub/internetmarketing,https://habrastorage.org/files/50e/211/9a0/50e...,@jasiejames,https://habrahabr.ru/users/jasiejames,"<img src=""https://habrastorage.org/files/50e/2...",3.496508
4,https://habrahabr.ru/post/190088/,2014-09-04T00:32:00.000Z,"['python', 'flask', 'mongodb', 'pet-project']",Thunderargs: практика использования. Часть 2,Программирование,История создания Часть 1 Добрый день. Вкратце...,Данияр Супиев,hub/programming,https://habrahabr.ru/i/habralogo.jpg,@uthunderbird,https://habrahabr.ru/users/uthunderbird,"<a href=""http://habrahabr.ru/post/223041/"">Ист...",3.688879


In [ ]:
data.shape

(172913, 13)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172913 entries, 0 to 172912
Data columns (total 13 columns):
_id             172913 non-null object
date            172913 non-null object
tags            172913 non-null object
title           172913 non-null object
hubs_title      172913 non-null object
description     172299 non-null object
name            143259 non-null object
hub             172913 non-null object
png             172913 non-null object
nick            135210 non-null object
url             172913 non-null object
content         172861 non-null object
favs_lognorm    172913 non-null float64
dtypes: float64(1), object(12)
memory usage: 17.1+ MB


In [ ]:
#уберём сэмплы, у которых пустые значения в колонке content, поскольку их очень мало.

data.dropna(subset = ['content'], axis = 0, inplace = True)

In [ ]:
data.shape

(172861, 13)

In [ ]:
#выделим год и месяц в отдельные колонки

data['date'] = data['date'].apply(pd.to_datetime)
data['year'] = data['date'].apply(lambda x: x.year)
data['month'] = data['date'].apply(lambda x: x.month)

In [ ]:
data['year'].unique()

array([2008, 2016, 2010, 2015, 2014, 2009, 2012, 2011, 2013, 2007, 2006])

In [ ]:
data.groupby('year')['month'].nunique()

year
2006    11
2007    12
2008    12
2009    12
2010    12
2011    12
2012    12
2013    12
2014    12
2015    12
2016    12
Name: month, dtype: int64

In [ ]:
#теперь посмотрим, за какие месяцы и годы доступна тестовая выборка

test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/test_content.csv')

In [ ]:
test.shape

(5405, 12)

In [ ]:
test['date'] = test['date'].apply(pd.to_datetime)
test['year'] = test['date'].apply(lambda x: x.year)
test['month'] = test['date'].apply(lambda x: x.month)

In [ ]:
test['year'].unique()

array([2017])

In [ ]:
test.groupby('year')['month'].unique()

year
2017    [1, 4, 2, 3]
Name: month, dtype: object

In [ ]:
#тестовые данные доступны за первые четыре месяца 2017 года
#для обучения модели будем использовать весь 2015 год в качестве трейна + 4 первые месяца 2016 года в качестве валидации

Xtr = data[data['year'] == 2015]
Xval = data[(data['year'] == 2016) & (data['month'] <= 4)]
ytr = Xtr['favs_lognorm']
yval = Xval['favs_lognorm']
Xtr.shape, Xval.shape

((23425, 15), (7556, 15))

In [ ]:
data = pd.concat([Xtr, Xval], axis = 0, ignore_index = True)

In [ ]:
data.shape

(30981, 15)

In [ ]:
stemmer = SnowballStemmer("russian")

In [ ]:
#функция для очистки текста:
#1)оставляем только русские и латинские символы
#2)к готовому набору стоп-слов вручную добавляем другие ненужные (см. ниже). Без них качество становится лучше
#3)удаляем ссылки
#4)делаем стемминг для русских слов

bad_symbols = re.compile("[^а-яА-Яa-zA-Z]")
stopwords = set(stopwords.words("english")) | set(stopwords.words("russian"))
remove_links = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

for i in ['img', 'src', 'href', 'image', 'align', 'right', 'left', 'br', '\r\n', 'div', 'style', 'text', 'center', 'ul', 'li', 'alt', 'title', 'width', 'height', 'name', 'h', 'b', 'e', 'см', 'кликабельно', 'картинка']:
  stopwords.add(i)

def clean_text(text):
    text = remove_links.sub(' ', text)
    text = bad_symbols.sub(' ', text)
    text = text.lower().split()
    text = [stemmer.stem(word) for word in text if word not in stopwords]
    
    return text

In [ ]:
data['clean_content'] = data['content'].apply(clean_text)

In [ ]:
#колонка с очищенным контентом

data['clean_content']

0        [pricing, большинств, предпринимател, сосредот...
1        [фот, нас, сегодн, зонд, dawn, заверш, сво, ле...
2        [жизн, самоучк, счита, fortra, pl, котор, уч, ...
3        [десятилет, развит, электрон, коммерц, сша, со...
4        [sad, girl, lambd, expression, пар, месяц, наз...
5        [спрос, сво, жен, план, выходн, врод, ответ, р...
6        [лаборатор, инвитр, запуст, октокоптер, пробир...
7        [сайт, интерет, магазин, newegg, com, опублико...
8        [пользу, разн, вид, смарт, карт, повседневн, ж...
9        [font, color, black, сто, лет, назад, ноябр, г...
10       [эт, стат, метод, диагностик, почтов, протокол...
11       [робот, med, отличн, компаньон, дет, врем, про...
12       [хоч, рассказа, сво, участ, побед, ежегодн, со...
13       [сегодняшн, ден, далек, кажд, довол, используе...
14       [немецк, автоконцерн, daimler, американск, раз...
15       [друз, гот, кругл, календар, кажд, год, рис, н...
16       [эт, истор, начина, публикац, сам, сложн, крос.

In [ ]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [ ]:
MAX_NB_WORDS = 5000

MAX_SEQUENCE_LENGTH = 300

EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words = MAX_NB_WORDS, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower = True)
tokenizer.fit_on_texts(data['clean_content'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 312006 unique tokens.


In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
proc = tokenizer.texts_to_sequences(data['clean_content'].values)
proc = pad_sequences(proc, maxlen = MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', proc.shape)

Shape of data tensor: (30981, 300)


In [ ]:
x_train = proc[:23425, :]
x_val = proc[23425:, :]
y_train = ytr.values
y_val = yval.values

In [ ]:
x_train.shape, x_val.shape

((23425, 300), (7556, 300))

In [ ]:
#для начала обучим простую линейную модель

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(x_train, y_train)
train_preds = model.predict(x_train)
valid_preds = model.predict(x_val)

In [ ]:
#данный показатель будем использовать в качестве бейслайна против нейронной сети

print('Train error', mean_squared_error(y_train, train_preds))
print('Validation error', mean_squared_error(y_val, valid_preds))

Train error 1.3046205379074725
Validation error 1.2549024180717707


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length = x_train.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(128, dropout = 0.3, recurrent_dropout = 0.3))
model.add(Dense(1, activation = 'linear'))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          500000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 300, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 617,377
Trainable params: 617,377
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 10
batch_size = 128

history = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size,
                    validation_data = (x_val, y_val), callbacks = [EarlyStopping(monitor = 'val_loss', patience = 2, min_delta = 0.001)])

Instructions for updating:
Use tf.cast instead.
Train on 23425 samples, validate on 7556 samples
Epoch 1/10
23425/23425 [==============================] - 120s 5ms/step - loss: 2.2055 - val_loss: 1.0020
Epoch 2/10
23425/23425 [==============================] - 117s 5ms/step - loss: 0.9855 - val_loss: 0.8823
Epoch 3/10
23425/23425 [==============================] - 117s 5ms/step - loss: 0.8292 - val_loss: 0.7980
Epoch 4/10
23425/23425 [==============================] - 116s 5ms/step - loss: 0.7619 - val_loss: 0.8166
Epoch 5/10
23425/23425 [==============================] - 116s 5ms/step - loss: 0.7304 - val_loss: 0.7847
Epoch 6/10
23425/23425 [==============================] - 115s 5ms/step - loss: 0.6835 - val_loss: 0.8001
Epoch 7/10
23425/23425 [==============================] - 117s 5ms/step - loss: 0.6457 - val_loss: 0.8277


In [ ]:
#качество значительно возросло в сравнении с регрессионной моделью